In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional,TimeDistributed, concatenate
from keras.layers import GlobalMaxPool1D, GlobalAvgPool1D, Masking
from keras.models import Model,Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import TensorBoard
from keras import metrics

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import scipy
import sys

In [3]:
from process_data import load_data,get_rank_of_size
from layers import AttentionLayer
from data_util import unpack_news_data,DATE_INTERVAL_NEWS,MAX_NEWS_NUM,EMBEDDING_SIZE

In [4]:
rank = get_rank_of_size()

In [5]:
data = load_data(rank[:4])

data of FB is loaded
data of AAPL is loaded
data of GOOGL is loaded
data of AMZN is loaded


In [7]:
data['GOOGL']['train'][0][0][0].toarray().shape

(50, 768)

# baseline model

In [22]:
def build_baseline_model(code='Default'):
    news_input = Input(shape=(DATE_INTERVAL_NEWS,MAX_NEWS_NUM,EMBEDDING_SIZE))
    day_layer = Bidirectional(GRU(20,return_sequences=True))
    inv_layer = Bidirectional(GRU(20,return_sequences=True))
    
    x = TimeDistributed(day_layer)(news_input)
    x = TimeDistributed(Dropout(0.5))(x)
    x = TimeDistributed(AttentionLayer())(x)
    
    x = inv_layer(x)
    x = Dropout(0.5)(x)
    x = AttentionLayer()(x)
    
    x = Dense(2, activation='softmax')(x)
    model = Model(inputs=news_input,outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [23]:
model = build_baseline_model()

In [24]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 7, 50, 768)        0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 7, 50, 40)         94680     
_________________________________________________________________
time_distributed_11 (TimeDis (None, 7, 50, 40)         0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 7, 40)             41        
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 7, 40)             7320      
_________________________________________________________________
dropout_9 (Dropout)          (None, 7, 40)             0         
_________________________________________________________________
attention_layer_7 (Attention (None, 40)                41        
__________

In [25]:
for i in range(1):
    model.fit(x=unpack_news_data(data[rank[i]]['train'][0]),y=data[rank[i]]['train'][2],\
              batch_size=32,epochs=40,verbose=1,\
    validation_data=(unpack_news_data(data[rank[i]]['val'][0]),data[rank[i]]['val'][2]),\
    callbacks=[TensorBoard(log_dir='model_log/')])

(1143, 7, 50, 768)
(56, 7, 50, 768)
Train on 1143 samples, validate on 56 samples
Epoch 1/40
1143/1143 [==============================] - 10s 8ms/step - loss: 1.4326 - acc: 0.5188 - val_loss: 0.8934 - val_acc: 0.3929
Epoch 2/40
1143/1143 [==============================] - 8s 7ms/step - loss: 1.0611 - acc: 0.5276 - val_loss: 1.1341 - val_acc: 0.4286
Epoch 3/40
1143/1143 [==============================] - 8s 7ms/step - loss: 0.9508 - acc: 0.5398 - val_loss: 1.6303 - val_acc: 0.4107
Epoch 4/40
 384/1143 [=========>....................] - ETA: 5s - loss: 0.8220 - acc: 0.5807

KeyboardInterrupt: 

In [ ]:
model.evaluate(x=x_test, y=y_test,batch_size=16)

# NUMERICAL MODEL

In [ ]:
numerical_timestep = 20 #  correspond to the 'size' of  the window
attribute_num = 5 # Open/High/Low/AdjClose/Volume 

def build_numerical_model(code='Default'):
    numerical_input = Input(shape=(numerical_timestep,attribute_num))
    x = GRU(50,return_sequences=True)(numerical_input)
    x = Dropout(0.4)(x)
    x = GRU(50)(x)
    x = Dropout(0.4)(x)
    x = Dense(10)(x)
    x = Dense(2,activation='softmax')(x)
    model = Model(inputs=numerical_input,outputs=x)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
nmodel = build_numerical_model()

In [ ]:
print(nmodel.summary())

In [ ]:
nmodel.fit(x=data['AMZN']['train'][1],y=data['AMZN']['train'][2],batch_size=16,epochs=50,verbose=1,\
          validation_data=(data['AMZN']['val'][1],data['AMZN']['val'][2]),\
          callbacks=[TensorBoard(log_dir='num_log/')])